In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pycaret

     |████████████████████████████████| 256kB 13.4MB/s 
     |████████████████████████████████| 157.5MB 50kB/s 
     |████████████████████████████████| 245kB 53.3MB/s 
     |████████████████████████████████| 14.2MB 65.4MB/s 
     |████████████████████████████████| 102kB 14.8MB/s 
     |████████████████████████████████| 6.8MB 55.2MB/s 
     |████████████████████████████████| 276kB 66.7MB/s 
     |████████████████████████████████| 174kB 72.8MB/s 
     |████████████████████████████████| 65.8MB 111kB/s 
     |████████████████████████████████| 1.8MB 56.3MB/s 
     |████████████████████████████████| 1.6MB 53.4MB/s 
     |████████████████████████████████| 604kB 57.4MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 81kB 13.8MB/s 
     |████████████████████████████████| 3.1MB 64.7MB/s 
     |████████████████████████████████| 81kB 13.8MB/s 
     |████████████████████████████████| 163kB 61.5MB/s 
     |████████████████████████████████| 153kB 62.7M

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from pycaret.classification import *

In [ ]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

In [ ]:
path = '/content/drive/Shareddrives/dacon/404/preprocessed/mk2_with_categorized/'

test_err_42_counts = pd.read_csv(path + '테스트_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])
train_err_42_counts = pd.read_csv(path + '트레인_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])

test_err_code_frq = pd.read_csv(path + '테스트_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])
train_err_code_frq = pd.read_csv(path +'트레인_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])

test_err_sum_byday = pd.read_csv(path +'테스트_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])
train_err_sum_byday = pd.read_csv(path +'트레인_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])

test_qui_frq = pd.read_csv(path +'테스트_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])
train_qui_frq = pd.read_csv(path +'트레인_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])

test_qui_sum_byday = pd.read_csv(path +'테스트_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])
train_qui_sum_byday = pd.read_csv(path +'트레인_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])

In [ ]:
# 각 사용자가 42가지 에러타입을 몇 번씩 겪었는지 나타냄
train_err_42_counts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


In [ ]:
# 각 사용자가 가장 자주 겪은 에러코드를 나타냄(위에는 에러타입, 여기는 에러코드)
train_err_code_frq

,Most_Err_Code,Second_Err_Code
10000,1,0
10001,1,standby
10002,1,0
10003,1,0
10004,1,connection timeout
...,...,...
24995,1,0
24996,1,ETC
24997,1,0
24998,1,0


In [ ]:
# 각 사용자가 퀼리티 1 부터 퀼리티 12까지 자주 겪은 퀼리티 로그를 나타냄
# _1은 가장 자주 겪은 로그이며, _2는 두 번째로 자주 겪은 로그임. (None은 2번째 로그가 없음, 한 가지 오류만 겪음)
train_qui_frq

,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q7_1,q7_2,q8_1,q8_2,q9_1,q9_2,q10_1,q10_2,q11_1,q11_2,q12_1,q12_2
10000,0.0,None,0.0,None,0.0,4.0,0.0,None,0.0,None,0.0,None,0.0,None,7.0,4.0,0.0,None,0.0,None
10002,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,None,0.0,1.0,3.0,5.0,0.0,-1.0,0.0,None
10004,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,None,0.0,None,1.0,3.0,0.0,-1.0,0.0,None
10005,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,None,0.0,None,4.0,5.0,0.0,-1.0,0.0,None
10006,0.0,None,0.0,None,0.0,4.0,0.0,1.0,0.0,1.0,0.0,None,0.0,None,4.0,None,0.0,None,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,1.0,None,0.0,-1.0,0.0,None
24992,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,5.0,3.0,0.0,-1.0,0.0,None
24993,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,5.0,3.0,0.0,-1.0,0.0,None
24995,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,3.0,2.0,0.0,-1.0,0.0,None


In [ ]:
# Error 데이터를 일일 단위로 쪼개서 각 사용자가 날짜별로 겪은 에러코드의 누계
# 0일은 10월 31일, 마지막은 11월 30일
train_err_sum_byday

,er_sum_0,er_sum_1,er_sum_2,er_sum_3,er_sum_4,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30
10000,0,11,20,38,43,53,62,82,89,94,104,118,123,140,149,165,174,182,199,218,223,230,235,240,245,254,265,277,295,308,317
10001,0,11,61,90,138,180,209,222,246,299,351,374,413,421,462,486,501,531,556,583,628,653,671,689,716,727,785,2237,2275,2337,2365
10002,0,10,23,36,51,60,68,85,96,108,118,129,140,145,160,173,188,193,202,217,222,232,248,251,257,268,273,289,289,297,306
10003,0,9,23,33,38,54,59,61,72,77,96,120,126,132,134,136,151,157,164,179,184,191,212,223,240,246,254,275,286,300,306
10004,0,25,46,95,123,134,162,180,209,229,283,302,323,340,360,375,390,408,449,457,559,587,603,621,644,671,690,714,732,755,777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,41,58,76,76,100,103,133,155,172,194
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
24997,0,9,43,77,141,163,194,218,248,274,292,301,320,339,377,408,418,445,464,496,525,550,579,621,654,695,718,748,779,813,826
24998,0,7,7,17,17,24,24,32,32,39,76,83,83,89,91,96,102,102,111,112,118,120,125,131,131,137,137,144,144,151,155


In [ ]:
# Quality 데이터를 일일 단위로 쪼개서 퀼리티 로그가 사용자 별로 몇 번 수집되었나
# 누적, 마지막 11월 30일 자료는 최종 몇 번의 로그 수집이 있었는가를 의미함.
train_qui_sum_byday

,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
10002,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0
10004,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
24992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
24993,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
24995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0


In [ ]:
train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

train_err3 = train_err2[['user_id', 'model_nm']]
test_err3 = test_err2[['user_id', 'model_nm']]

train_err3 = train_err3.drop_duplicates(subset = ['user_id'])
test_err3 = test_err3.drop_duplicates(subset = ['user_id'])

train_err3.set_index('user_id', inplace = True)
test_err3.set_index('user_id', inplace = True)

In [ ]:
# train_err_42_counts, train_err_code_frq, train_qui_frq, train_err_sum_byday, train_qui_sum_byday
# test_err3, test_err_42_counts, test_err_code_frq, test_qui_frq, test_err_sum_byday, test_qui_sum_byday
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
test_qui_frq = test_qui_frq.astype(str)
train_qui_frq = train_qui_frq.astype(str)

In [ ]:
testset = pd.concat([test_err3, test_err_42_counts, test_err_sum_byday, test_qui_sum_byday], axis = 1)
trainset = pd.concat([train_err3, train_err_42_counts, train_err_sum_byday, train_qui_sum_byday], axis = 1)
testset = testset.fillna(0)
trainset = trainset.fillna(0)
trainset.iloc[:,1:] = scaler.fit_transform(trainset.iloc[:,1:])
testset.iloc[:,1:] = scaler.transform(testset.iloc[:,1:])

In [ ]:
trainset = pd.concat([trainset, train_err_code_frq, train_qui_frq], axis = 1)
testset = pd.concat([testset, test_err_code_frq, test_qui_frq], axis = 1)

In [ ]:
trainset2 = trainset.reset_index()
testset2 = testset.reset_index()

In [ ]:
probs = list(set(train_prob.user_id))

In [ ]:
trainset2['TARGET'] = trainset2['index'].apply(lambda x: 1 if x in probs else 0)
trainset2.set_index('index', inplace=True)

In [ ]:
X = trainset2.iloc[:,:-1]
y = pd.DataFrame(trainset2.iloc[:,-1])
X

,model_nm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,er_sum_0,er_sum_1,er_sum_2,er_sum_3,er_sum_4,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30,Most_Err_Code,Second_Err_Code,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q7_1,q7_2,q8_1,q8_2,q9_1,q9_2,q10_1,q10_2,q11_1,q11_2,q12_1,q12_2
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,model_3,0.0,0.0,0.001098,0.000533,0.000000,0.000557,0.000372,0.0,0.0,0.000143,0.002047,0.002182,0.000172,0.000728,0.038663,0.039869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135021,0.000010,0.000010,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000762,0.000912,0.001733,0.001485,0.001782,0.002044,0.002167,0.002350,0.002351,0.002174,0.002280,0.002197,0.002496,0.002635,0.002473,0.002376,0.002341,0.002377,0.002399,0.002260,0.001874,0.001739,0.001683,0.001216,0.001183,0.001222,0.001270,0.001342,0.001401,0.001422,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002941,0.005682,1,0,0.0,None,0.0,None,0.0,4.0,0.0,None,0.0,None,0.0,None,0.0,None,7.0,4.0,0.0,None,0.0,None
10001,model_2,0.0,0.0,0.000000,0.000000,0.007986,0.000557,0.000372,0.0,0.0,0.000000,0.001364,0.001500,0.000172,0.001092,0.098952,0.083660,0.000000,0.097561,0.043478,0.130435,0.045455,0.047416,0.028345,0.001745,0.012821,0.092827,0.000000,0.000000,0.0,0.00,0.006644,0.0,0.054645,0.001457,0.0,0.166667,0.166667,0.000000,0.0,0.066667,0.041698,0.030303,0.0,0.000762,0.002783,0.004105,0.004765,0.006051,0.006889,0.005866,0.006496,0.007478,0.007336,0.007227,0.007376,0.007505,0.008170,0.007284,0.006841,0.006830,0.006641,0.006415,0.006364,0.005321,0.004965,0.004832,0.003554,0.003385,0.003621,0.010258,0.010348,0.010630,0.010640,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,standby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,model_3,0.0,0.0,0.000275,0.000676,0.000151,0.001114,0.000372,0.0,0.0,0.000020,0.001774,0.001909,0.000172,0.000291,0.034076,0.033987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000698,0.000000,0.105485,0.000021,0.000021,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000693,0.001049,0.001642,0.001761,0.002017,0.002241,0.002246,0.002535,0.002701,0.002466,0.002493,0.002500,0.002585,0.002829,0.002593,0.002567,0.002482,0.002413,0.002388,0.002250,0.001890,0.001835,0.001760,0.001276,0.001248,0.001259,0.001325,0.001315,0.001351,0.001373,0.0,0.0,0.000000,0.000000,0.020833,0.016667,0.027778,0.02381,0.020833,0.018519,0.016667,0.022727,0.021898,0.020134,0.018634,0.023121,0.021622,0.020305,0.023923,0.027149,0.025751,0.024490,0.023346,0.022305,0.021352,0.020478,0.019672,0.018927,0.018293,0.017647,0.022727,1,0,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,None,0.0,1.0,3.0,5.0,0.0,-1.0,0.0,None
10003,model_2,0.0,0.0,0.000000,0.000000,0.000301,0.000557,0.000372,0.0,0.0,0.000000,0.001228,0.

In [ ]:
testset2.set_index('index', inplace = True)

In [ ]:
clf = setup(data = trainset2, target = 'TARGET') 

,Description,Value
0,session_id,7915
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(15000, 128)"
5,Missing Values,True
6,Numeric Features,102
7,Categorical Features,25
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7838,0.8153,0.5018,0.7823,0.6113,0.4713,0.4936,8.698
lightgbm,Light Gradient Boosting Machine,0.7826,0.8078,0.5125,0.7693,0.6150,0.4719,0.4909,0.287
gbc,Gradient Boosting Classifier,0.7759,0.8081,0.4673,0.7843,0.5855,0.4451,0.4732,2.857
rf,Random Forest Classifier,0.7702,0.7997,0.4619,0.7678,0.5766,0.4319,0.4583,1.208
xgboost,Extreme Gradient Boosting,0.7693,0.7898,0.5268,0.7176,0.6074,0.4495,0.4604,31.948
et,Extra Trees Classifier,0.7678,0.7880,0.4940,0.7337,0.5903,0.4368,0.4535,0.990
ada,Ada Boost Classifier,0.7669,0.7913,0.4732,0.7466,0.5791,0.4289,0.4503,0.640
lda,Linear Discriminant Analysis,0.7499,0.7503,0.3829,0.7601,0.5090,0.3649,0.4036,0.207
ridge,Ridge Classifier,0.7470,0.0000,0.3711,0.7598,0.4983,0.3549,0.3958,0.037
svm,SVM - Linear Kernel,0.7440,0.0000,0.3840,0.7392,0.5014,0.3527,0.3887,0.209


In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7814,0.8125,0.4944,0.7805,0.6053,0.4645,0.4877
1,0.7738,0.8046,0.4916,0.7559,0.5957,0.4483,0.4684
2,0.7852,0.8161,0.5021,0.7863,0.6129,0.4741,0.4969
3,0.7800,0.8078,0.4866,0.7810,0.5997,0.4590,0.4835
4,0.7832,0.8177,0.5120,0.7712,0.6154,0.4729,0.4922
Mean,0.7807,0.8117,0.4973,0.7750,0.6058,0.4638,0.4857
SD,0.0039,0.0049,0.0089,0.0107,0.0075,0.0095,0.0098


In [ ]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8027,0.8208,0.5198,0.7937,0.6281,0.5017,0.5225


In [ ]:
final_model = finalize_model(blended)

In [ ]:
cat = create_model('catboost')
tuned_cat = tune_model(cat, optimize='AUC')

In [ ]:
# 앙상블로 예측하고 싶으면 model = final_model, 캣부로 해보고 싶으면 model = tuned_cat
model = tuned_cat

predictions = predict_model(model, data = testset2)

In [ ]:
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i+30000] ==1:
    x.append(predictions['Score'][i+30000])
  else:
    x.append(1-predictions['Score'][i+30000])

In [ ]:
sample_submssion = pd.read_csv('/content/drive/Shareddrives/dacon/404/sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("CAT.csv", index = False)
!cp CAT.csv "drive/My Drive/"